# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

1. The objective is to identify how the different features of a used car ( **independent varibales** ) affect the price ( **dependent variable** )

2. Since the dependent variable is continuous, meaning the price value can be any number, this is considered a **Regression** analysis

3. Also, given there is a specific well-known dependent variable, this is will be a **Supervised** learning process

4. Lastly, as the ultimate goal is to maximize the price, this ML excercise is **Exploitative** in nature

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

1. Load the necessary libraries to load the dataset and perform initial analysis

In [38]:
import numpy as np
import pandas as pd

2. Load dataset

In [4]:
df = pd.read_csv('./data/vehicles.csv')

3. Check the contents of the dataframe

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

Initial Observations
* Most of the dependent variables are discrete
* Several features (columns) have incomplete indexes (rows)
* A few columns should be irrelevant (i.e. id, VIN)

4. Evaluating imputation for missing data

In [62]:
for i in df.columns:
    print(i,': \t' if len(i) > 7 else '\t\t',round((df[i].isna().sum()/len(df))*100,2),'%')

id 		 0.0 %
region 		 0.0 %
price 		 0.0 %
year 		 0.28 %
manufacturer : 	 4.13 %
model 		 1.24 %
condition : 	 40.79 %
cylinders : 	 41.62 %
fuel 		 0.71 %
odometer : 	 1.03 %
title_status : 	 1.93 %
transmission : 	 0.6 %
VIN 		 37.73 %
drive 		 30.59 %
size 		 71.77 %
type 		 21.75 %
paint_color : 	 30.5 %
state 		 0.0 %


5. Check option values on the different features

In [11]:
df['region'].value_counts()

region
columbus                   3608
jacksonville               3562
spokane / coeur d'alene    2988
eugene                     2985
fresno / madera            2983
                           ... 
meridian                     28
southwest MS                 14
kansas city                  11
fort smith, AR                9
west virginia (old)           8
Name: count, Length: 404, dtype: int64

This is a very large and convoluted classification where multiple cities are merged into a single option. Also this feature should be only relevant to each state individually. Given this includes data for the entire US, It may be best to rely on the 'state' feature to assess the effect of the car's location in the price

In [12]:
df['year'].value_counts()

year
2017.0    36420
2018.0    36369
2015.0    31538
2013.0    30794
2016.0    30434
          ...  
1943.0        1
1915.0        1
1902.0        1
1905.0        1
1909.0        1
Name: count, Length: 114, dtype: int64

The year model should be more appropriate as an integer and it may be bes to reclassify this as the 'age' of the car to have a better scaling with relation to other values. Also, there is 1% missing values for this feature which should be imputable without major concern for introducing strong bias

In [13]:
df['manufacturer'].value_counts()

manufacturer
ford               70985
chevrolet          55064
toyota             34202
honda              21269
nissan             19067
jeep               19014
ram                18342
gmc                16785
bmw                14699
dodge              13707
mercedes-benz      11817
hyundai            10338
subaru              9495
volkswagen          9345
kia                 8457
lexus               8200
audi                7573
cadillac            6953
chrysler            6031
acura               5978
buick               5501
mazda               5427
infiniti            4802
lincoln             4220
volvo               3374
mitsubishi          3292
mini                2376
pontiac             2288
rover               2113
jaguar              1946
porsche             1384
mercury             1184
saturn              1090
alfa-romeo           897
tesla                868
fiat                 792
harley-davidson      153
ferrari               95
datsun                63
aston-martin

This is a very detailed classification with too many value options. It may be best to reclassify this as European, American, Japanese, Korean. Also, noticed Harley-Davidson manufacturer, which only manufactured utility cars (i.e golf carts) hence this dataset may include motorcycles, which would be a complete different dynamic when assessing the price. This feature has 5% missing data which should be imputable without major concern for introducing strong bias

In [14]:
df['model'].value_counts()

model
f-150                      8009
silverado 1500             5140
1500                       4211
camry                      3135
silverado                  3023
                           ... 
plymouth fury 1               1
f550 box truck delivery       1
duramax 2500hd                1
3 s sport                     1
Paige Glenbrook Touring       1
Name: count, Length: 29649, dtype: int64

The 'model' is also a very specific classification mostly relevant within the same brand and it should not have strong correlation across brands for a fair comparison. Using body 'type' would be more effective (i.e. truk, suv, sedan, etc.). There is 2% missing data for this feature but it most likely be ignored.

In [15]:
df['condition'].value_counts()

condition
good         121456
excellent    101467
like new      21178
fair           6769
new            1305
salvage         601
Name: count, dtype: int64

This feature is expected to have a high positive correlation with price and it should be possible to tranform it using Ordinal Encoding. The main concern is that there is 40% of missing data for this feature, which is a high percentage to be imputed. It may be necessary to run Multiple Imputation

In [27]:
df['cylinders'].value_counts()

cylinders
6 cylinders     94169
4 cylinders     77642
8 cylinders     72062
5 cylinders      1712
10 cylinders     1455
other            1298
3 cylinders       655
12 cylinders      209
Name: count, dtype: int64

Given the number of value options, this feature may just increase the complexity of the model, specially if using Polynomial Transformation, and not provide a strong correlation, either negative or positive, to the price. Also, since there is 41% missing data for this feature, it should likely be ignored to reduce the Multiple Imputation process and the model's complexity

In [17]:
df['fuel'].value_counts()

fuel
gas         356209
other        30728
diesel       30062
hybrid        5170
electric      1698
Name: count, dtype: int64

This feature is also expected to have a strong correlation to the price. There is < 1% missing data for this feature so there should be no problem to impute it without any concern for introducing strong bias

In [18]:
df['odometer'].value_counts()

odometer
100000.0    2263
1.0         2246
0.0         1965
200000.0    1728
150000.0    1603
            ... 
149468.0       1
154259.0       1
111887.0       1
213826.0       1
26892.0        1
Name: count, Length: 104870, dtype: int64

This feature is also expected to have a high correlation with the price. As this is a continuous variable, it may be used as is otherwise it could be reclassified per milage range (i.e. 25K, 50K, 100K, 150K, 200K+). There is a 1% missing data for this feature so there should be no problem to impute it without any concern for introducing strong bias

In [19]:
df['title_status'].value_counts()

title_status
clean         405117
rebuilt         7219
salvage         3868
lien            1422
missing          814
parts only       198
Name: count, dtype: int64

This feature should also be strongly correlated with the price. There is 2% missind data for this feature so there should be no problem to impute it without any concern for introducing strong bias

In [28]:
df['transmission'].value_counts()

transmission
automatic    336524
other         62682
manual        25118
Name: count, dtype: int64

This feature should have some positive correlation with the price. There is < 1% missind data for this feature so there should be no problem to impute it without any concern for introducing strong bias

In [21]:
df['drive'].value_counts()

drive
4wd    131904
fwd    105517
rwd     58892
Name: count, dtype: int64

This feature should have some positive correlation with the price. Given there is 31% missing data for this feature, it should likely be ignored to reduce the Multiple Imputation process

In [29]:
df['size'].value_counts()

size
full-size      63465
mid-size       34476
compact        19384
sub-compact     3194
Name: count, dtype: int64

This feature should have strong positive correlation with the price. Despite the expected importance, there is 72% missing data therefore this feature should be ignored

In [23]:
df['type'].value_counts()

type
sedan          87056
SUV            77284
pickup         43510
truck          35279
other          22110
coupe          19204
hatchback      16598
wagon          10751
van             8548
convertible     7731
mini-van        4825
offroad          609
bus              517
Name: count, dtype: int64

This feature should also be highly correlated with the price. The main concern is that there is 22% of missing data for this feature, which is a high percentage to be imputed. It may be necessary to run Multiple Imputation Also, the effect of other features ('size' & 'model') that will likely be ignored may be correlated with this feature

In [24]:
df['paint_color'].value_counts()

paint_color
white     79285
black     62861
silver    42970
blue      31223
red       30473
grey      24416
green      7343
custom     6700
brown      6593
yellow     2142
orange     1984
purple      687
Name: count, dtype: int64

This feature is expected to have some correlation with the price. Given there is 31% missing data for this feature, it should likely be ignored to reduce the Multiple Imputation process

In [25]:
df['state'].value_counts()

state
ca    50614
fl    28511
tx    22945
ny    19386
oh    17696
or    17104
mi    16900
nc    15277
wa    13861
pa    13753
wi    11398
co    11088
tn    11066
va    10732
il    10387
nj     9742
id     8961
az     8679
ia     8632
ma     8174
mn     7716
ga     7003
ok     6792
sc     6327
mt     6294
ks     6209
in     5704
ct     5188
al     4955
md     4778
nm     4425
mo     4293
ky     4149
ar     4038
ak     3474
la     3196
nv     3194
nh     2981
dc     2970
me     2966
hi     2964
vt     2513
ri     2320
sd     1302
ut     1150
wv     1052
ne     1036
ms     1016
de      949
wy      610
nd      410
Name: count, dtype: int64

The state should have a very strong correlation witht the prices but given the large number of value options, this feature may increase significantly the complexity of the data set

'VIN' and 'id' will be ignored as those should be irrelevant

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.